In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.interpolate import spline


#定义预设变量
k = 3 #曲线次数，k=3即，选区3次曲线
m = 34 #m值为通过的型值点的个数
n = m + k - 1#控制顶点数
nj = n + k + 1 #节点矢量总个数,如果通过的数据点数是n，则控制点数为n+2， 节点矢量为控制点数+次数+1，即，n+k+3
hx = k + 1 #常将两端点处重复度取为k+1


#px = np.array([0.0000, 0.0212, 0.0212, 0.0425, 0.0425, 0.0850, 0.0850, 0.1275, 0.1275, 0.17, 0.17, 0.255, 0.255, 0.34, 0.34, 0.425, 0.425, 0.51, 0.51, 0.68, 0.68, 0.85, 0.85, 1.02, 1.02, 1.19, 1.19, 1.36, 1.36, 1.53, 1.53, 1.615, 1.615, 1.655, 1.7 ])
#py = np.array([0.0000, 0.0454, -0.0209, 0.0614, -0.0291, 0.0835, -0.0384, 0.0986, -0.0444, 0.1093, -0.0496, 0.1222, -0.0595, 0.1275, -0.0675, 0.1292,-0.0727, 0.1284, -0.0758, 0.1214, -0.0761, 0.1090, -0.0709, 0.0969, -0.0624, 0.0741, -0.0510, 0.0524,-0.0367, 0.0286, -0.0209, 0.0156, -0.0119, 0.0167, 0.0000])

#px = np.array([-80.0, -40.0, -20.0, -10.0, 30.0, 50.0, 70.0])
#py = np.array([-60.0, 20.0, 10.0, 20.0, -20.0, 40.0, 30.0])

#测试数据
px = np.array([8.0, 8.0215, 8.0237, 8.0445, 8.0471, 8.0820, 8.0879, 8.1262, 8.1298, 8.1746, 8.1727, 8.2567, 8.2558, 8.3437, 8.3479, 8.4271, 8.4290, 8.5145, 8.5127, 8.6828, 8.6856, 8.8529, 8.8573, 9.0278, 9.0269, 9.195, 9.1982, 9.3686, 9.3697, 9.5356, 9.5329, 9.613, 9.6179, 9.6548, 9.7149])
py = np.array([0.0, 0.0463, -0.0298, 0.0648, -0.0226, 0.0858, -0.0338, 0.0946, -0.0427, 0.1046, -0.0494, 0.1223, -0.0565, 0.1256, -0.0642, 0.1262, -0.0776, 0.1283, -0.0779, 0.1291, -0.0767, 0.1090, -0.0737, 0.0949, -0.0628, 0.0749, -0.0574, 0.0539, -0.0349, 0.0244, -0.0210, 0.0130, -0.0130, 0.0172, 0.0])


print('X data size is: ', px.size)
print('Y data size is: ', py.size)

#已经计算出的控制定点坐标
dx = np.array([0.0000, 0.0060, 0.0180, 0.0493, 0.0846, 0.1275, 0.1841, 0.2549, 0.3399, 0.4249, 0.5383, 0.6800, 0.8501, 1.0199, 1.1908, 1.3576, 1.5090, 1.6025, 1.7489, 1.6024, 1.5091, 1.3577, 1.1905, 1.0205, 0.8499, 0.6802, 0.5384, 0.4250, 0.3399, 0.2546, 0.1835, 0.1269, 0.0836, 0.0485, 0.0129, 0.0043, 0.0000])
dy = np.array([0.0000, -0.0073, -0.0220, -0.0312, -0.0390, -0.0444, -0.0514, -0.0598, -0.0680, -0.0731, -0.0772, -0.0771, -0.0713, -0.0629, -0.0516, -0.0369, -0.0239, -0.0123, -0.0010, 0.0165, 0.0325, 0.0532, 0.0728, 0.1002, 0.1077, 0.1228, 0.1282, 0.1297, 0.1280, 0.1235, 0.1132, 0.0991, 0.0842, 0.0655, 0.0429, 0.0143, 0.0000])

print(dx.size)               
print(dx.size)

np.set_printoptions(precision=4)   #设置矩阵显示数据的显示精度

#定义型值点
Q = []
for i in range(m+1):
    Q.append((px[i], py[i]))
    
print('Q列表的值为：')
print(Q)

S = 0
num = 0

#定义节点矢量数组
U = [] 
L = []  #存储计算的每段的弦长



    
#计算总弦长
for i in range(1, m+1):
    x = px[i] - px[i-1]
    y = py[i] - py[i-1]
    l = math.sqrt(x*x + y*y)
    #print('the every length is :', l)
    L.append(l)
    S = S + l
    num = num + 1
    
print('各段线段长度列表为： ')
print(L)
#print(len(L))
print('总线段长度为：')
print(S)

#计算从k+1-n的其余节点矢量，3次的计算方法
def countUDirect(n, k, S, L, U):
    nu = 0
    #添加前k+1个节点矢量
    for a in range(k+1):
        U.append(0.0)
    
    for j in range(k+1, n+1):
        a = (k+1)/2 + nu
        l = L[0:int(a)-1]
        print('每个l的值', l)
        u = sum(l) / S
        U.append(u)
        nu = nu + 1
        print('单个节点和计算： ', sum(l))
        print('单个节点矢量计算：', u)
        
    #增加最后k+1）个节点矢量
    for k in range(k+1):
        U.append(1.0)



countUDirect(n, k, S, L, U)
print('节点向量列表为：')
print(U)

#定义算子
# O = []   
# for b in range(0, n+k):
#     O.append(U[b+1]-U[b])    
    
# print('算子列表值：')
# print(O)   
# print('算子列表个数：')
# print(len(O))


 
PX = np.mat(px).T
PY = np.mat(py).T

print('PX is :')
#print(PX)
print('PY is :')
#print(PY)


A = []
B = []
C = []
E = []
F = []
    
#求解控制点矩阵替代值
for i in range(0, n+k-3):
    if (U[i+3]-U[i] == 0.0):
        A.append(0.0)
    else:
        A.append(((U[i+3]-U[i+2])*(U[i+3]-U[i+2]))/(U[i+3]-U[i]))
        
    if ((U[i+4]-U[i+1] == 0.0) and (U[i+3]-U[i] == 0.0)):
        B.append(0.0)
        
    if ((U[i+4]-U[i+1] == 0.0) and (U[i+3]-U[i] != 0.0)):
        B.append(((U[i+3]-U[i+2])*(U[i+2]-U[i]))/(U[i+3]-U[i]))
        
    if((U[i+3]-U[i] == 0) and (U[i+4]-U[i+1] != 0.0)):
        B.append(((U[i+2]-U[i+1])*(U[i+4]-U[i+2]))/(U[i+4]-U[i+1]))
        
    if ((U[i+4]-U[i+1] != 0.0) and (U[i+3]-U[i] != 0.0)):
        B.append((((U[i+3]-U[i+2])*(U[i+2]-U[i]))/(U[i+3]-U[i])) + (((U[i+2]-U[i+1])*(U[i+4]-U[i+2]))/(U[i+4]-U[i+1])))
    
    if (U[i+4]-U[i+1] == 0.0):
        C.append(0.0)
    else:
        C.append(((U[i+2]-U[i+1])*(U[i+2]-U[i+1]))/(U[i+4]-U[i+1]))
    #E.append((U[i+3]-U[i+1])*px[i-1])
    #F.append((U[i+3]-U[i+1])*py[i-1])

for i in range(1, n+k-3):
    E.append((U[i+3]-U[i+1])*px[i-1])
    F.append((U[i+3]-U[i+1])*py[i-1])

print('U4 is :', U[4])
print('U3 is :', U[3])

print('E0 is :', E[0])
print('F0 is :', F[0])


print('A的个数为：', len(A))
print(A)
print('B的个数为：', len(B))
print(B)
print('C的个数为：', len(C))
print(C)

#构造m介的单位矩阵,并设置对角线及偏移对角线上的元素
#D = np.eye(m)
# print('D is :')
# print(D)
# for i in range(m):
#     D[i,i] = B[i]
    
# for i in range(m-1):
#     D[i, i+1] = C[i]
    
# for i in range(m-1):
#     D[i+1, i] = A[i+1]


#设置两个端点处的值
#D[0, m-1] = A[0]
#D[m-1, 0] = C[m-1]
    
# print('After set value B is :')
# print(D)
#print('*********************************************')
#用Scipy求逆矩阵，linalg.inv()
#DT = linalg.inv(D)
#print('D-1 :')
#print(DT)

#计算闭合曲线的系数矩阵，并计算控制顶点

dd = np.eye(n-2)

print('dd is :')
print(dd)
print('dd shape is ', dd.shape)

#进行矩阵的赋值


for i in range(0, n-2):
    dd[i,i] = B[i+1]
    
for i in range(0, n-3):
    dd[i+1, i] = A[i+2]
    
for i in range(0, n-3):
    dd[i, i+1] = C[i+1]
    
dd[0, n-3] = A[1]
dd[n-3, 0] = C[n-2]

print('A is ')
print(A)
print('B is ')
print(B)
print('C is ')
print(C)
    
print('进行矩阵赋值后的DD为：')
print(dd)
print(dd.shape)
print('**********************************************************************')
print('E is ')
print(E)
print(np.array(E).shape)
print(np.array(F).shape)

eex = []
for i in range(1, n-1):
    eex.append(E[i])
eey = []
for i in range(1, n-1):
    eey.append(F[i])

eex = np.array(eex)
eey = np.array(eey)

print('E matrix is ', eex)
print('F matrix is ', eey)

print('求dd的逆矩阵为：')
#print(linalg.inv(dd))

wd_x = linalg.solve(dd,eex)
WDX = []
WDX.append(px[0])
for i in range(n-2):
    WDX.append(wd_x[i])
WDX.append(WDX[1])
WDX.append(WDX[2])

print('处理前的wdx为： ')
print(wd_x)
print('求解后的控制顶点X坐标为：')
print(WDX)
print('控制顶点的维数：', np.array(WDX).shape)
wd_y = linalg.solve(dd, eey)
WDY = []
WDY.append(py[0])
for i in range(n-2):
    WDY.append(wd_y[i])
WDY.append(WDY[1])
WDY.append(WDY[2])

print('处理前的wd_y为： ')
print(wd_y)
print('求解后的控制顶点Y坐标为：')
print(WDY)
print('控制顶点的维数为：', np.array(WDY).shape)

#测试计算开放曲线的d0计算
print('N 的个数是：', n)

ka = np.eye(n-1)
print('ka is :')
print(ka)
print('*************************************************************')
print('A')
print(A)
print('B')
print(B)
print('C')
print(C)
# print('E is ')
# print(E)
# print('F is ')
# print(F)

ka[0,0] = 1.0
ka[0,2] = 0.0
ka[0,3] = 0.0

for i in range(1, n-2):
    ka[i, i] = B[i+1]
    
for i in range(0, n-2):
    ka[i, i+1] = C[i+1]
    
for i in range(0, n-3):
    ka[i+1, i] = A[i+2]
    
ka[n-2, n-2] = 1.0
ka[n-2, n-3] = 0.0
ka[n-2, n-4] = 0.0

print('进行转换后的ka为：')
print(ka)
print(ka.shape)

#进行两个端点切失条件的计算, 这里两个端点的切失计算应该还是不对的，需要修正
#d0x = px[0]  + (U[4]-U[3])/3 * (math.sqrt((px[1]-px[0])*(px[1]-px[0]) + (py[1]-py[0])*(py[1]-py[0])) / (U[4]-U[3]))
d0 = math.sqrt((px[1]-px[0])*(px[1]-px[0]) + (py[1]-py[0])*(py[1]-py[0])) / (U[4]-U[3])
#d0 = (px[1] - px[0])/(U[4]-U[3])
#d0y = py[0]  + (U[4]-U[3])/3 * (math.sqrt((px[1]-px[0])*(px[1]-px[0]) + (py[1]-py[0])*(py[1]-py[0])) / (U[4]-U[3]))

#dnx = px[n-2] - ((U[n+1]-U[n]) / 3) * (math.sqrt((px[6]-px[5])*(px[6]-px[5]) + (py[6]-py[5]) * (py[6]-py[5]))) / (U[n+1] - U[n])
#dny = py[n-2] - ((U[n+1]-U[n]) / 3) * (math.sqrt((px[6]-px[5])*(px[6]-px[5]) + (py[6]-py[5]) * (py[6]-py[5]))) / (U[n+1] - U[n])
dn = math.sqrt((px[n-2]-px[n-3])*(px[n-2]-px[n-3]) + (py[n-2]-py[n-3]) * (py[n-2]-py[n-3])) / (U[n+1] - U[n])


print('d0x is ')
print(d0)
print('d0y is ')
print(d0)
print('dnx is ')
print(dn)
print('dny is ')
print(dn)

e1x = px[0] + ((U[4]-U[3])/3)*d0
e1y = py[0] + ((U[4]-U[3])/3)*d0
print('e1x is ')
print(e1x)
print('e1y is ')
print(e1y)

en1x = px[n-2] - ((U[n+1]-U[n])/3)*dn
en1y = py[n-2] - ((U[n+1]-U[n])/3)*dn
print('en1x is ')
print(en1x)
print('en1y is ')
print(en1y)
#***********************************************************************


print('E的值为：')
print(E)
print('E的长度为：')
print(len(E))
EX = []
for i in range(1, n-1):
    EX.append(E[i])
#EX.append(px[m])
EX[0] = e1x
EX[n-3] = en1x
arr_EX =np.array(EX)

print('EX is :')
print(arr_EX)
print('EX shape is ', arr_EX.shape)




#DX = linalg.solve(ka, arr_EX)
print('DX solution is :')
print(len(DX))

DXX = []
dxl = DX.tolist()
print('dxl is :')
print(dxl)
print('DX is :')
print(DX)


DXX.append(px[0])
for i in range(0, n-1):
    DXX.append(dxl[i])
DXX.append(px[m])


EY = []

for i in range(1, n-1):
    EY.append(F[i])
#EY.append(py[m])
EY[0] = e1y
EY[n-3] = en1y
arr_EY = np.array(EY)

print('EY is :')
print(EY)
print('EY shape is ', arr_EY.shape)

#DY = linalg.solve(ka, arr_EY)
print('DY solution is :')
print(DY)
print(len(DY))

DYY = []
dyl = DY.tolist()
DYY.append(py[0])
for i in range(0, n-1):
    DYY.append(dyl[i])
DYY.append(py[m])


print('DXX and DYY is :')
# for i in range(n+1):
#     (DXX[i], DYY[i])
print(DXX)
print(DYY)


#用numpy求解逆矩阵，D.I, 首先要将ndarray转换成矩阵，即np.mat(),然后使用mat.I来计算逆矩阵
#DH = np.mat(D)
#print('使用Numpy求解的逆矩阵是：')
#print(DH.I)

# print('型值点的X坐标为：')
# print(px)

# print('型值点的Y坐标为：')
# print(py)

# Dx = linalg.solve(D,E)
# print('X值的个数：')
# print(len(Dx))
# print('控制顶点的X坐标为：')
# print(Dx)
# Dy = linalg.solve(D,F)
# print('Y值的个数：')
# print(len(Dy))
# print('控制顶点的Y坐标为：')
# print(Dy)

# for i in range(len(Dx)):
#     print('控制点坐标为：')
#     print(Dx[i], Dy[i])

fig = plt.figure()
ax = plt.axes()

#plt.plot(px, py, 'o', color='red')
#plt.plot(px, py, '-p', color='red')
#plt.scatter(px, py, color='blue')
plt.scatter(px, py, color='green', linestyle='dashed')
#plt.plot(dx, dy, marker='p', color='blue')
#plt.scatter(WDX, WDY, marker='o', color='red')
#plt.plot(DXX, DYY, color='red', linestyle='solid')

# p12x = px[0] - px[1]
# p12y = py[0] - py[1]
# p12 = math.sqrt(p12x*p12x + p12y*p12y)
# print('P1P2=', p12)

# p23x = px[2] - px[3]
# p23y = py[2] - py[3]
# p23 = math.sqrt(p23x*p23x + p23y*p23y)
# print('P2P3=', p23)




    
print('总长度S= ', S)
#print('第一段节点向量值为：', p12/S)
#print('第二段节点向量值为：', (p12+p23)/S)
print('循环次数为：', num)


print('*****************************************************')
print('节点矢量的值为：')
print(np.array(U))
print('节点矢量的数量为：', len(U))
print('n的数量：')
print(n)

print('d0 is ', d0)
print('dn is ', dn)

print('头节点： ', (U[4]-U[3]))
print('尾节点：', (U[n+1]-U[n]))

plt.show()


X data size is:  35
Y data size is:  35
37
37
Q列表的值为：
[(8.0, 0.0), (8.0215, 0.0463), (8.0237, -0.0298), (8.0445, 0.0648), (8.0471, -0.0226), (8.082, 0.0858), (8.0879, -0.0338), (8.1262, 0.0946), (8.1298, -0.0427), (8.1746, 0.1046), (8.1727, -0.0494), (8.2567, 0.1223), (8.2558, -0.0565), (8.3437, 0.1256), (8.3479, -0.0642), (8.4271, 0.1262), (8.429, -0.0776), (8.5145, 0.1283), (8.5127, -0.0779), (8.6828, 0.1291), (8.6856, -0.0767), (8.8529, 0.109), (8.8573, -0.0737), (9.0278, 0.0949), (9.0269, -0.0628), (9.195, 0.0749), (9.1982, -0.0574), (9.3686, 0.0539), (9.3697, -0.0349), (9.5356, 0.0244), (9.5329, -0.021), (9.613, 0.013), (9.6179, -0.013), (9.6548, 0.0172), (9.7149, 0.0)]
各段线段长度列表为： 
[0.051048408398303505, 0.07613179362132486, 0.09685969233897028, 0.08743866421669538, 0.11387962943388964, 0.11974543832647648, 0.13399048473679057, 0.13734718781249217, 0.15396210572735108, 0.15401172033322658, 0.19114625290598805, 0.17880226508632377, 0.20220489608315595, 0.18984646428100782, 0.206215

NameError: name 'DX' is not defined

In [12]:
print(L)
print(L[0:2])
print(sum(L))

[89.44271909999159, 22.360679774997898, 14.142135623730951, 56.568542494923804, 63.245553203367585, 22.360679774997898]
[89.44271909999159, 22.360679774997898]
268.12030997200975


In [1]:
f = [(-80,60), (-40,20), (-20, 10), (-10,20), (30,-20), (50,40), (70, 30)]
print(f)

[(-80, 60), (-40, 20), (-20, 10), (-10, 20), (30, -20), (50, 40), (70, 30)]


In [6]:
print(f[1][0])

-40


In [10]:
import numpy as np
from scipy import linalg

In [16]:
m, n = 5, 6
A = np.random.rand(m,m)
B = np.random.rand(m,n)
X1 = linalg.solve(A, B)
X2 = np.dot(linalg.inv(A), B)
#print(np.allclose(X1, X2))
print(A)
print(B)
print('After compute the linalg: ')
print(X1)
print(X2)

%timeit linalg.solve(A, B)
%timeit np.dot(linalg.inv(A), B)

[[0.9742 0.58   0.9787 0.6754 0.9808]
 [0.7678 0.0458 0.7435 0.7129 0.1061]
 [0.1394 0.2966 0.5996 0.6634 0.7185]
 [0.1805 0.0591 0.1019 0.6314 0.2008]
 [0.6482 0.7274 0.303  0.7252 0.7039]]
[[0.5715 0.4296 0.7773 0.5566 0.8172 0.9038]
 [0.1991 0.658  0.5301 0.849  0.7578 0.2785]
 [0.909  0.4253 0.5434 0.587  0.6688 0.8519]
 [0.7142 0.6685 0.3972 0.6945 0.0533 0.4707]
 [0.1144 0.3876 0.0366 0.4641 0.5967 0.5019]]
After compute the linalg: 
[[ 0.0613  0.4251  0.6001  0.1701 -0.7665  0.0751]
 [-2.4363 -1.047  -2.0074 -0.676   1.7686 -1.0704]
 [-0.6954 -0.484  -0.2008 -0.0529  1.6159 -0.231 ]
 [ 0.7796  0.9702  0.2227  1.0914  0.2761  0.4066]
 [ 2.1196  0.4499  1.4306  0.0995 -1.254   1.4304]]
[[ 0.0613  0.4251  0.6001  0.1701 -0.7665  0.0751]
 [-2.4363 -1.047  -2.0074 -0.676   1.7686 -1.0704]
 [-0.6954 -0.484  -0.2008 -0.0529  1.6159 -0.231 ]
 [ 0.7796  0.9702  0.2227  1.0914  0.2761  0.4066]
 [ 2.1196  0.4499  1.4306  0.0995 -1.254   1.4304]]
47.1 µs ± 944 ns per loop (mean ± std. dev. 

In [21]:
aa = [1,2,3,4,6,8]
bb = np.array(aa)
#print(aa)
bb
cc = bb.tolist()
cc

[1, 2, 3, 4, 6, 8]

In [8]:
ppx = np.array([200.0, 300.0, 450.0, 600.0])
ppy = np.array([300.0, 200.0, 400.0, 270.0])

In [12]:
DP = []
L = 0
UP = []
for i in range(3):
    DP.append(math.sqrt((ppx[i+1]-ppx[i])*(ppx[i+1]-ppx[i]) + (ppy[i+1]-ppy[i])*(ppy[i+1]-ppy[i])))
    L = L + DP[i]
    
UP.append(0.0)
UP.append(DP[0]/L)
UP.append((DP[0]+DP[1])/L)
UP.append(1.0)
print(DP)
print('所有弦长的和为：')
print(L)
print('节点矢量为：')
print(UP)


[141.4213562373095, 250.0, 198.4943324127921]
所有弦长的和为：
589.9156886501016
节点矢量为：
[0.0, 0.2397314717310242, 0.66352084504312, 1.0]


In [25]:
k = 3 #曲线次数，k=3即，选区3次曲线
m = 10 #m值为通过的型值点的个数
n = m + k - 1#控制顶点数

qz = np.array([0.0, 15.0, 65.0, 115.0, 165.0, 216.0, 265.0, 315.0, 365.0, 417.0, 432.0])


S = 0
num = 0

#定义节点矢量数组
U = [] 
L = []  #存储计算的每段的弦长



    
#计算总弦长
for i in range(1, m+1):
    #x = px[i-1] - px[i]
    #y = py[i-1] - py[i]
    #l = math.sqrt(x*x + y*y)
    l = abs(qz[i-1] - qz[i])
    print('the every length is :', l)
    L.append(l)
    S = S + l
    num = num + 1
    
#计算从k+1-n的其余节点矢量，3次的计算方法
def countUDirect(n, k, S, L, U):
    nu = 0
    #添加前k+1个节点矢量
    for a in range(k+1):
        U.append(0.0)
    
    for j in range(k+1, n+1):
        a = (k+1)/2 + nu
        l = L[0:int(a)-1]
        print('每个l的值', l)
        u = sum(l) / S
        U.append(u)
        nu = nu + 1
        print('单个节点和计算： ', sum(l))
        print('单个节点矢量计算：', u)
        
    #增加最后k+1）个节点矢量
    for k in range(k+1):
        U.append(1.0)

countUDirect(n, k, S, L, U)
print('节点矢量为： ')
print(U)
print('U的维数为：')
print(np.array(U).shape)

the every length is : 15.0
the every length is : 50.0
the every length is : 50.0
the every length is : 50.0
the every length is : 51.0
the every length is : 49.0
the every length is : 50.0
the every length is : 50.0
the every length is : 52.0
the every length is : 15.0
每个l的值 [15.0]
单个节点和计算：  15.0
单个节点矢量计算： 0.034722222222222224
每个l的值 [15.0, 50.0]
单个节点和计算：  65.0
单个节点矢量计算： 0.15046296296296297
每个l的值 [15.0, 50.0, 50.0]
单个节点和计算：  115.0
单个节点矢量计算： 0.2662037037037037
每个l的值 [15.0, 50.0, 50.0, 50.0]
单个节点和计算：  165.0
单个节点矢量计算： 0.3819444444444444
每个l的值 [15.0, 50.0, 50.0, 50.0, 51.0]
单个节点和计算：  216.0
单个节点矢量计算： 0.5
每个l的值 [15.0, 50.0, 50.0, 50.0, 51.0, 49.0]
单个节点和计算：  265.0
单个节点矢量计算： 0.6134259259259259
每个l的值 [15.0, 50.0, 50.0, 50.0, 51.0, 49.0, 50.0]
单个节点和计算：  315.0
单个节点矢量计算： 0.7291666666666666
每个l的值 [15.0, 50.0, 50.0, 50.0, 51.0, 49.0, 50.0, 50.0]
单个节点和计算：  365.0
单个节点矢量计算： 0.8449074074074074
每个l的值 [15.0, 50.0, 50.0, 50.0, 51.0, 49.0, 50.0, 50.0, 52.0]
单个节点和计算：  417.0
单个节点矢量计算： 0.9652777777777778
节点矢量为：

In [3]:
from scipy.interpolate import interp1d

In [4]:
x = np.linspace(0, 10, num=11, endpoint=True)
y = np.cos(-x**2/9.0)
f = interp1d(x, y)
f2 = interp1d(x, y, kind='cubic')


In [ ]:
xnew = np.linspace(0, 10, num=41, endpoint=True)
import matplotlib.pyplot as plt
plt.plot(x, y, 'o', xnew, f(xnew), '-', xnew, f2(xnew), '--')
plt.legend(['data', 'linear', 'cubic'], loc='best')
plt.show()
